In [1]:
import openai
import pandas as pd
import json
from IPython.display import Markdown
import sys

In [2]:
zz=openai.Model.list()

In [3]:
len(zz['data'])

62

In [4]:
[zz['data'][idx]['id'] for idx in range(len(zz['data'])) if 'gpt' in zz['data'][idx]['id']]

['gpt-4',
 'gpt-4-0613',
 'gpt-4-0314',
 'gpt-3.5-turbo-16k-0613',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo-0301',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo']

In [5]:
model_params = {
    'model': "gpt-4",
    'temperature': 0,
}

In [6]:
def print_save(string, file='../cmp2.md'):
    print(string)
    with open(file,'w') as f:
        f.write(string)

## Load `prompt_template.md`

In [7]:
sys.path.append('..')
from utils import *
from functools import partial

In [8]:
# prompt_dict=load_prompt_template('../prompt_template.md')
prompt_dict = partial(load_prompt_template,file='../prompt_template.md')

In [30]:
prompt_dict().keys()

dict_keys(['Preamble', 'Construct Kinetic Hamiltonian (continuum version)', 'Construct Kinetic Hamiltonian (lattice version)', 'Define each term in Kinetic Hamiltonian (continuum version)', 'Construct Potential Hamiltonian (continuum version)', 'Define each term in Potential Hamiltonian (continuum version)', 'Construct interaction Hamiltonian (real space, lattice version)', 'Construct interaction Hamiltonian (momentum space)', 'Convert from single-particle to second-quantized form, return in matrix', 'Convert from single-particle to second-quantized form, return in summation (expand the matrix)', 'Convert noninteracting Hamiltonian in real space to momentum space (continuum version)', 'Convert noninteracting Hamiltonian in real space to momentum space (lattice version)', 'Convert interacting Hamiltonian in real space to momentum space (lattice version)', 'Particle-hole transformation', 'Simplify the Hamiltonian in the particle-hole basis', "Wick's theorem", 'Extract quadratic term', 'H

In [9]:
print(prompt_dict['Construct Kinetic Hamiltonian (continuum version)'])

You will be instructed to describe the kinetic term of Hamiltonian in {system} in the {real|momentum} space in the {single-particle|second-quantized} form.   
The degrees of freedom of the system are: {dof}  
Express the Kinetic Hamiltonian {kinetic_symbol} using {var} which are only on the diagonal terms, and arrange the basis in the order of {order}.

Use the following conventions for the symbols:  
{def_var}




In [11]:
df=pd.DataFrame(list(prompt_dict.items()),columns=['Task','Prompt'])

## Construct Kinetic Hamiltonian (continuum version, single-particle)

In [382]:
user_msg=[]
AI_msg=[]
sys_msg=[{'role': 'system', 'content': prompt_dict()['Preamble']}]
kwargs_list=[]

In [383]:
kwargs = {
    'task':    'Construct Kinetic Hamiltonian (continuum version, single-particle)',
    'system':    'the hole-doped AB-stacked MoTe2/WSe2',
    'real|momentum':    'real',
    'single-particle|second-quantized':    'single-particle',
    'dof':    'valley index (+K and -K valley), layer index (top and bottom layer)',
    'kinetic_symbol':    r'$H_{Kinetic}(r)$',
    'var':    r'$E_{\tau,l}$',
    'order':    '(+K,bottom), (+K,top), (-K,bottom), (-K,top)',
    'Note that the sublattice degrees of freedom is suppressed for now and will be stated later|None': '',
    'def_var':
    r'''$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{\tau,l}$ : energy dispersion for layer $l$ and valley $\tau$''',
    'answer':    '$H_{Kinetic}(r) = \\begin{pmatrix} E_{+K,b} & 0 & 0 & 0 \\\\ 0 & E_{+K,t} & 0 & 0 \\\\ 0 & 0 & E_{-K,b} & 0 \\\\ 0 & 0 & 0 & E_{-K,t} \\end{pmatrix}$'
}


In [384]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to describe the kinetic term of Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in the real space in the single-particle form.   
The degrees of freedom of the system are: valley index (+K and -K valley), layer index (top and bottom layer).  
Express the Kinetic Hamiltonian $H_{Kinetic}(r)$ using $E_{\tau,l}$ which are only on the diagonal terms, and arrange the basis in the order of (+K,bottom), (+K,top), (-K,bottom), (-K,top). 

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{\tau,l}$ : energy dispersion for layer $l$ and valley $\tau$


In [385]:
# user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [386]:
# messages=assembly_message(sys_msg,user_msg,AI_msg)

In [387]:
# rs = openai.ChatCompletion.create(messages=messages, **model_params)

# Markdown(rs['choices'][0]['message'].content)

In [388]:
# AI_msg.append(dict(rs['choices'][0]['message']))

In [389]:
# rs['choices'][0]['message'].content

In [390]:
kwargs_list.append(kwargs)

## Define each term in Kinetic Hamiltonian (continuum version)

In [391]:
# user_msg=[]
# AI_msg=[]
# sys_msg=[{'role': 'system', 'content': prompt_dict()['Preamble']}]
# kwargs_list=[]

In [392]:
kwargs = {
    'task': 'Define each term in Kinetic Hamiltonian (continuum version)',
    'var':r' four $E_{\tau,l}(k)$',
    'Description':r'''For all energy dispersions, $E_{l,\tau}(k)$, it characterizes the dispersion for free holes, namely, $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$.   
In addition, for the top layer, $E_{\tau,t}(k)$, there is a shift of momentum $k$, to make the band bottom at $\tilde{\tau}\kappa$. You should infer the correct $\tilde{\tau}$.  
Finally, in the real space, the momentum $k=-i \partial_r$. You should keep the form of $k$ in the Hamiltonian for short notations but should remember $k$ is an operator.  ''',
    'expression_kinetic': '$H_{Kinetic}(r) = \\begin{pmatrix} E_{+K,b} & 0 & 0 & 0 \\\\ 0 & E_{+K,t} & 0 & 0 \\\\ 0 & 0 & E_{-K,b} & 0 \\\\ 0 & 0 & 0 & E_{-K,t} \\end{pmatrix}$',
    'kinetic_symbol': r'$H_{Kinetic}(r)$',
    'def_var':r'''$k$ : momentum operator  
$r$ : real space operator  
$m_l$ : the effective mass for layer $l$  
$\tilde{\tau}$ : takes value of +1 for $\tau=+K$ and -1 for $\tau=-K$  
$\pm\kappa$ : corners of monolayer moire Brillouin zone  ''',
'answer':'$H_{Kinetic}(r) = \\begin{pmatrix} -\\frac{\\hbar^2 k^2}{2 m_b} & 0 & 0 & 0 \\\\ 0 & -\\frac{\\hbar^2 (k-\\kappa)^2}{2 m_t} & 0 & 0 \\\\ 0 & 0 & -\\frac{\\hbar^2 k^2}{2 m_b} & 0 \\\\ 0 & 0 & 0 & -\\frac{\\hbar^2 (k+\\kappa)^2}{2 m_t} \\end{pmatrix}$'
}

In [393]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to construct each term, namely  four $E_{\tau,l}(k)$.  
For all energy dispersions, $E_{l,\tau}(k)$, it characterizes the dispersion for free holes, namely, $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$.   
In addition, for the top layer, $E_{\tau,t}(k)$, there is a shift of momentum $k$, to make the band bottom at $\tilde{\tau}\kappa$. You should infer the correct $\tilde{\tau}$.  
Finally, in the real space, the momentum $k=-i \partial_r$. You should keep the form of $k$ in the Hamiltonian for short notations but should remember $k$ is an operator.  
You should recall that $H_{Kinetic}(r) = \begin{pmatrix} E_{+K,b} & 0 & 0 & 0 \\ 0 & E_{+K,t} & 0 & 0 \\ 0 & 0 & E_{-K,b} & 0 \\ 0 & 0 & 0 & E_{-K,t} \end{pmatrix}$.   
Return the expression for  four $E_{\tau,l}(k)$ in the Kinetic Hamiltonian, and substitute it into the Kinetic Hamiltonian $H_{Kinetic}(r)$.  

Use the following conventions for the symbols (You should also obey the conventions in all my previous prom

In [394]:
# user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [395]:
# messages=assembly_message(sys_msg,user_msg,AI_msg)

In [396]:
# rs = openai.ChatCompletion.create(messages=messages, **model_params)
# Markdown(rs['choices'][0]['message'].content)

In [397]:
# AI_msg.append(dict(rs['choices'][0]['message']))

In [398]:
# rs['choices'][0]['message'].content

In [399]:
kwargs_list.append(kwargs)

## Construct Potential Hamiltonian (continuum version)

In [400]:
# user_msg=[]
# AI_msg=[]
# sys_msg=[{'role': 'system', 'content': prompt_dict['Preamble']}]
# kwargs_list=[]

In [401]:
kwargs = {
    'task':
    'Construct Potential Hamiltonian (continuum version)',
    'potential_symbol':'$H_{Potential}(r)$',
    'real|momentum':'real',
    'single-particle|second-quantized':'single-particle',
    'kinetic_symbol': r'$H_{Kinetic}(r)$',
    
    'Description': r'''At each valley, it is a 2 by 2 matrix $H_{Potential,\tau}(r)$, where each component is layer index. The diagonal term in this 2 by 2 matrix $H_{Potential,\tau}(r)$ is  intralayer tunneling $\Delta_l(r)$. The off-diagonal term in this 2 by 2 matrix $H_{Potential,\tau}(r)$ is interlayer potential $\Delta_{T,\tau}(r)$, which should be kept hermitian.  
There is no hybridization between $+K$ valley and $-K$ valley, i.e., the corresponding matrix elements in $H_{Potential}(r)$ are all zero''',
    'var':
    r'$\Delta_l(r)$ and $\Delta_{T,\tau}(r)$',
    'def_var': r'''  
$\Delta_l$: intralayer potential for layer index $l$  
$\Delta_{T,\tau}$ : interlayer tunneling at valley index $\tau$  
$H_{Potential}(r)$ : potential term of Hamiltonian in the real space in the single-particle form.  ''',
    'answer':'$H_{Potential}(r) = \\begin{pmatrix} \\Delta_b(r) & \\Delta_{T,+K}(r) & 0 & 0 \\\\ \\Delta_{T,+K}^*(r) & \\Delta_t(r) & 0 & 0 \\\\ 0 & 0 & \\Delta_b(r) & \\Delta_{T,-K}(r) \\\\ 0 & 0 & \\Delta_{T,-K}^*(r) & \\Delta_t(r) \\end{pmatrix}$'
}


In [402]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to describe the potential term of Hamiltonian $H_{Potential}(r)$ in the real space in the single-particle form.  
The potential Hamiltonian has the same degrees of freedom as the kinetic Hamiltonian $H_{Kinetic}(r)$.  
At each valley, it is a 2 by 2 matrix $H_{Potential,\tau}(r)$, where each component is layer index. The diagonal term in this 2 by 2 matrix $H_{Potential,\tau}(r)$ is  intralayer tunneling $\Delta_l(r)$. The off-diagonal term in this 2 by 2 matrix $H_{Potential,\tau}(r)$ is interlayer potential $\Delta_{T,\tau}(r)$, which should be kept hermitian.  
There is no hybridization between $+K$ valley and $-K$ valley, i.e., the corresponding matrix elements in $H_{Potential}(r)$ are all zero.  
Express the potential Hamiltonian $H_{Potential}(r)$ using $\Delta_l(r)$ and $\Delta_{T,\tau}(r)$.  

Use the following conventions for the symbols (You should also remember the conventions in my previous prompts if there are no conflicts. If you have conflicts in 

In [403]:
# user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [404]:
# messages=assembly_message(sys_msg,user_msg,AI_msg)

In [405]:
# rs = openai.ChatCompletion.create(messages=messages, **model_params)

# Markdown(rs['choices'][0]['message'].content)

In [406]:
# AI_msg.append(dict(rs['choices'][0]['message']))

In [407]:
# rs['choices'][0]['message'].content

In [408]:
kwargs_list.append(kwargs)

## Define each term in Potential Hamiltonian (continuum version)

In [409]:
# user_msg=[]
# AI_msg=[]
# sys_msg=[{'role': 'system', 'content': prompt_dict['Preamble']}]
# kwargs_list=[]

In [410]:
kwargs = {
    'task':
    'Define each term in Potential Hamiltonian (continuum version)',
    'potential_symbol':'$H_{Potential}(r)$',
    'var':    r'the intralayer potential $\Delta_l(r)$ and interlayer tunneling $\Delta_{T,\tau}(r)$',
    'real|momentum':'real',
    'single-particle|second-quantized':'single-particle',
    'kinetic_symbol':    '$H_{Kinetic}(r)$',
    'Description': r'''The intralayer on the top layer $\Delta_t(r)$ is $V_{zt}$.  
The intralayer on the bottom layer $\Delta_b(r)$ is $2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b)$.  
The interlayer tunneling at valley $\tau$ is $\tilde{\tau}w(1+\omega^{\tilde{\tau}}e^{i\tilde{\tau} g_2 \cdot r} + \omega^{2\tilde{\tau}} e^{i\tilde{\tau} g_3 \cdot r})$. You should infer the correct $\tilde{\tau}$ such that $\tilde{\tau}$ does not appear in the answer.  ''',
    'def_var': r'''$V_b$ : bottom layer potential strength  
$\psi_b$ : bottom layer potential phase  
$g_j$ : reciprocal unit vector on the first shell corresponding in the moire Brillouin zone, j varies from 1 to 6.  
$r$ : real space position  
$\omega$ : phase factor to respect the threefold rotational symmetry, which is $e^{i2\pi/3}$, the hermitian conjugate is $\omega^{-1}=e^{-i2\pi/3}$  
$w$ : interlayer tunneling strength''',
    'expression_Potential': '$H_{Potential}(r) = \\begin{pmatrix} \\Delta_b(r) & \\Delta_{T,+K}(r) & 0 & 0 \\\\ \\Delta_{T,+K}^*(r) & \\Delta_t(r) & 0 & 0 \\\\ 0 & 0 & \\Delta_b(r) & \\Delta_{T,-K}(r) \\\\ 0 & 0 & \\Delta_{T,-K}^*(r) & \\Delta_t(r) \\end{pmatrix}$',
    'answer':'$H_{Potential}(r) = \\begin{pmatrix} 2V_b\\sum_{j=1,3,5} \\cos(g_j \\cdot r  +\\psi_b) & w(1+\\omega e^{i g_2 \\cdot r} + \\omega^2 e^{i g_3 \\cdot r}) & 0 & 0 \\\\ w(1+\\omega^{-1}e^{-i g_2 \\cdot r} + \\omega^{-2} e^{-i g_3 \\cdot r}) & V_{zt} & 0 & 0 \\\\ 0 & 0 & 2V_b\\sum_{j=1,3,5} \\cos(g_j \\cdot r  +\\psi_b) & -w(1+\\omega^{-1}e^{-i g_2 \\cdot r} + \\omega^{-2} e^{-i g_3 \\cdot r}) \\\\ 0 & 0 & -w(1+\\omega e^{i g_2 \\cdot r} + \\omega^2 e^{i g_3 \\cdot r}) & V_{zt} \\end{pmatrix}$'
}


In [411]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to construct each term $H_{Potential}(r)$, namely, the intralayer potential $\Delta_l(r)$ and interlayer tunneling $\Delta_{T,\tau}(r)$.  
The intralayer on the top layer $\Delta_t(r)$ is $V_{zt}$.  
The intralayer on the bottom layer $\Delta_b(r)$ is $2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b)$.  
The interlayer tunneling at valley $\tau$ is $\tilde{\tau}w(1+\omega^{\tilde{\tau}}e^{i\tilde{\tau} g_2 \cdot r} + \omega^{2\tilde{\tau}} e^{i\tilde{\tau} g_3 \cdot r})$. You should infer the correct $\tilde{\tau}$ such that $\tilde{\tau}$ does not appear in the answer.  .
You should recall that $H_{Potential}(r) = \begin{pmatrix} \Delta_b(r) & \Delta_{T,+K}(r) & 0 & 0 \\ \Delta_{T,+K}^*(r) & \Delta_t(r) & 0 & 0 \\ 0 & 0 & \Delta_b(r) & \Delta_{T,-K}(r) \\ 0 & 0 & \Delta_{T,-K}^*(r) & \Delta_t(r) \end{pmatrix}$.  
Return the expressions for the intralayer potential $\Delta_l(r)$ and interlayer tunneling $\Delta_{T,\tau}(r)$, and substitute it into the potential Hami

In [412]:
# user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [413]:
# messages=assembly_message(sys_msg,user_msg,AI_msg)

In [414]:
# rs = openai.ChatCompletion.create(messages=messages, **model_params)

# Markdown(rs['choices'][0]['message'].content)

In [415]:
# AI_msg.append(dict(rs['choices'][0]['message']))

In [416]:
# rs['choices'][0]['message'].content

In [417]:
kwargs_list.append(kwargs)

## Convert from single-particle to second-quantized form, return in matrix

In [418]:
kwargs = {
    'task':'Convert from single-particle to second-quantized form, return in matrix',
    'real|momentum':'real',
    'nonint_symbol': r'$H^{0}(r)$',
    'potential_symbol':'$H_{Potential}(r)$',
    'kinetic_symbol':'$H_{Kinetic}(r)$',
    '$r$|$k$': '$r$',
    'second_nonint_symbol': r'$\hat{H}^{0}$',
    'def_var': r'''$\psi_{\tau,l}(r)$ : electron annihilation operator at valley index $\tau$ and layer index $l$  
$\vec{\psi}(r)$ : spinor containing all annihilation operators $\psi_{\tau,l}(r)$ following the same order of $H^{0}(r)$  
$H^{0}(r)$ : single-particle noninteracting Hamiltonian in the real space  
$\hat{H}^{0}$ : total noninteracting Hamiltonian in the second quantized form ''',
    'answer':r'$\hat{H}^{0} = \int dr \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$'
}


In [419]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to construct the second quantized form of the total noninteracting Hamiltonian in the real space.  
The noninteracting Hamiltonian in the real space $H^{0}(r)$ is the sum of Kinetic Hamiltonian $H_{Kinetic}(r)$ and Potential Hamiltonian $H_{Potential}(r)$.  
To construct the second quantized form of a Hamiltonian. You should construct the creation and annihilation operators from the basis explicitly. You should follow the EXAMPLE below to convert a Hamiltonian from the single-particle form to second-quantized form.  
Finally by "total", it means you need to take a summation over the real space position $r$.   
Return the second quantized form of the total noninteracting Hamiltonian $\hat{H}^{0}$  

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
$\psi_{\tau,l}(r)

In [420]:
# user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [421]:
# messages=assembly_message(sys_msg,user_msg,AI_msg)

In [422]:
# rs = openai.ChatCompletion.create(messages=messages, **model_params)

# Markdown(rs['choices'][0]['message'].content)

In [423]:
# AI_msg.append(dict(rs['choices'][0]['message']))

In [424]:
# rs['choices'][0]['message'].content

In [425]:
kwargs_list.append(kwargs)

## Convert from single-particle to second-quantized form, return in summation (expand the matrix)

In [426]:
kwargs = {
    'task':'Convert from single-particle to second-quantized form, return in summation (expand the matrix)',
    'second_nonint_symbol':  r'$\hat{H}^{0}$',
    'matrix_element_symbol': r'$H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$',
    'basis_symbol': r'$\psi_{\tau,l}(r)$',
    'expression_second_nonint': r'$\hat{H}^{0} = \int dr \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$, where $H^{0}(r)=H_{Kinetic}(r)+H_{Potential}(r)$, $H_{Kinetic}(r) = \begin{pmatrix} -\frac{\hbar^2 k^2}{2 m_b} & 0 & 0 & 0 \\ 0 & -\frac{\hbar^2 (k-\kappa)^2}{2 m_t} & 0 & 0 \\ 0 & 0 & -\frac{\hbar^2 k^2}{2 m_b} & 0 \\ 0 & 0 & 0 & -\frac{\hbar^2 (k+\kappa)^2}{2 m_t} \end{pmatrix}$, and $H_{Potential}(r) = \begin{pmatrix} 2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b) & w(1+\omega e^{i g_2 \cdot r} + \omega^2 e^{i g_3 \cdot r}) & 0 & 0 \\ w(1+\omega^{-1}e^{-i g_2 \cdot r} + \omega^{-2} e^{-i g_3 \cdot r}) & V_{zt} & 0 & 0 \\ 0 & 0 & 2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b) & -w(1+\omega^{-1}e^{-i g_2 \cdot r} + \omega^{-2} e^{-i g_3 \cdot r}) \\ 0 & 0 & -w(1+\omega e^{i g_2 \cdot r} + \omega^2 e^{i g_3 \cdot r}) & V_{zt} \end{pmatrix}$',
    'def_var': r'''$H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$ : the matrix element of single-particle noninteracting Hamiltonian in the real space  ''',
    'answer':r'$\hat{H}^{0} = \int dr \sum_{l_1, l_2} \left( \psi_{+K,l_1}^\dagger(r) H^{0}_{+K,l_1,+K,l_2}(r) \psi_{+K,l_2}(r) + \psi_{-K,l_1}^\dagger(r) H^{0}_{-K,l_1,-K,l_2}(r) \psi_{-K,l_2}(r) \right)$'
}


In [427]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to expand the second-quantized form Hamiltonian $\hat{H}^{0}$ using $H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$ and $\psi_{\tau,l}(r)$. You should follow the EXAMPLE below to expand the Hamiltonian.  
You should use any previous knowledge to simplify it. For example, if any term of $H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$ is zero, you should remove it from the summation.
You should recall that $\hat{H}^{0} = \int dr \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$, where $H^{0}(r)=H_{Kinetic}(r)+H_{Potential}(r)$, $H_{Kinetic}(r) = \begin{pmatrix} -\frac{\hbar^2 k^2}{2 m_b} & 0 & 0 & 0 \\ 0 & -\frac{\hbar^2 (k-\kappa)^2}{2 m_t} & 0 & 0 \\ 0 & 0 & -\frac{\hbar^2 k^2}{2 m_b} & 0 \\ 0 & 0 & 0 & -\frac{\hbar^2 (k+\kappa)^2}{2 m_t} \end{pmatrix}$, and $H_{Potential}(r) = \begin{pmatrix} 2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b) & w(1+\omega e^{i g_2 \cdot r} + \omega^2 e^{i g_3 \cdot r}) & 0 & 0 \\ w(1+\omega^{-1}e^{-i g_2 \cdot r} + \omega^{-2} e^{-i g_3 \cdot r}) & V_{zt} & 0 & 0

In [428]:
kwargs_list.append(kwargs)

## Convert noninteracting Hamiltonian in real space to momentum space (continuum version)

In [429]:
kwargs = {
    'task':'Convert noninteracting Hamiltonian in real space to momentum space (continuum version)',
    'real_creation_op': r'$\psi_{\tau,l}^\dagger(r)$',
    'momentum_creation_op': r'$c_{\tau,l}^\dagger(k)$',
    'def_FT': r'$c_{\tau,l}^\dagger(k)= \frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}^\dagger(r) e^{i k \cdot r}$',
    'real_var': '$r$',
    'entire_real|fBZ': 'entire real space',
    'second_nonint_symbol':  r'$\hat{H}^{0}$',
    'def_var': r'''$c_{l,\tau}(k)$ : electron annihilation operator at momentum $k$, layer $l$, and valley $\tau$  
$c_{l,\tau}^\dagger(k)$ : electron creation operator at momentum $k$, layer $l$, and valley $\tau$  
$V$ : area of unit cell in the real space  ''',
    'answer':r'$\hat{H}^{0} = \sum_{\tau, l_1, l_2, k_1, k_2} c_{\tau,l_1}^\dagger(k_1) H^{0}_{\tau,l_1,\tau,l_2}(k_1,k_2) c_{\tau,l_2}(k_2)$'
}


In [430]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to convert the total noninteracting Hamiltonian in the second quantized form from the basis in real space to the basis by momentum space.  
To do that, you should apply the Fourier transformation to $\psi_{\tau,l}^\dagger(r)$ in the real space to the $c_{\tau,l}^\dagger(k)$ in the momentum space, which is defined as $c_{\tau,l}^\dagger(k)= \frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}^\dagger(r) e^{i k \cdot r}$, where $r$ is integrated over the entire real space. You should follow the EXAMPLE below to apply the Fourier transformation.  
Express the total noninteracting Hamiltonian $\hat{H}^{0}$ in terms of $c_{\tau,l}^\dagger(k)$. Simplify any summation index if possible.  

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
$c_{l,\tau}(k)$ : electron annihilation opera

In [431]:
kwargs_list.append(kwargs)

## Particle-hole transformation

In [432]:
kwargs = {
    'task':'Particle-hole transformation',
    'hole_op': r'$b_{l,\tau}(k)$',
    'particle_op': r'$c_{l,\tau}^\dagger(k)$',
    'particle_creation_op': r'$c_{l,\tau}$',
    'hole_creation_op': r'$b_{l,\tau}(k)^\dagger$',
    'particle_annihilation_op': r'$c_{l,\tau}^\dagger$',
    'hole_annihilation_op': r'$b_{l,\tau}(k)$',
    'second_nonint_symbol':  r'$\hat{H}^{0}$',
    'def_var': r'''$b_{l,\tau}(k)$ : hole annihilation operator at momentum $k$, layer $l$, and valley $\tau$''',
    'expression_particle_Ham' :r'$\hat{H}^{0} = \sum_{\tau, l_1, l_2, k_1, k_2} c_{\tau,l_1}^\dagger(k_1) H^{0}_{\tau,l_1,\tau,l_2}(k_1,k_2) c_{\tau,l_2}(k_2)$',
    'answer':r'$\hat{H}^{0} = \sum_{\tau, l_1, l_2, k_1, k_2} b_{\tau, l_1}(k_1) H^{0}_{\tau, l_1, \tau, l_2}(k_1, k_2) b_{\tau, l_2}^\dagger(k_2)$'
}


In [433]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to perform a particle-hole transformation.  
Define a hole operator, $b_{l,\tau}(k)$, which equals $c_{l,\tau}^\dagger(k)$.  
You should replace $c_{l,\tau}$ with $b_{l,\tau}(k)^\dagger$, and $c_{l,\tau}^\dagger$ with $b_{l,\tau}(k)$. You should follow the EXAMPLE below to apply the particle-hole transformation.  
You should recall that $\hat{H}^{0} = \sum_{\tau, l_1, l_2, k_1, k_2} c_{\tau,l_1}^\dagger(k_1) H^{0}_{\tau,l_1,\tau,l_2}(k_1,k_2) c_{\tau,l_2}(k_2)$.
Return the $\hat{H}^{0}$ in the hole operators.

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
$b_{l,\tau}(k)$ : hole annihilation operator at momentum $k$, layer $l$, and valley $\tau$

===  
EXAMPLE:  
Give a Hamiltonian  $\hat{H}=\sum_{k_1,k_2} c^\dagger(k_1) h(k_1,k_2) c(k_2)$ , and the particle-hol

In [434]:
kwargs_list.append(kwargs)

## Simplify the Hamiltonian in the particle-hole basis

In [435]:
kwargs = {
    'task':'Simplify the Hamiltonian in the particle-hole basis',
    'second_nonint_symbol':  r'$\hat{H}^{0}$',
    'hole_op': r'$b_{l,\tau}(k)$',
    'index_1': r'$l_1$/$k_1$',
    'index_2': r'$l_2$/$k_2$',
    'op': r'$b$',
    'Ham_op': r'$H$',
    'def_var': r'''$\delta_{i,j}$ : Kronecker delta function, take the value of 1 if $i=j$, and 0 otherwise.  
$`[b_i, b_j]_{+}`$ : anticommuting relation, i.e., $`[b_i, b_j]_+= b_i b_j +b_j b_i`$  
$\hat{H}^{0,final}$ : the final version of noninteracting Hamiltonian $\hat{H}^{0}$ in the hole basis after simplification.''',
    'expression_hole_Ham': r'$\hat{H}^{0} = \sum_{\tau, l_1, l_2, k_1, k_2} b_{\tau, l_1}(k_1) H^{0}_{\tau, l_1, \tau, l_2}(k_1, k_2) b_{\tau, l_2}^\dagger(k_2)$',
    'answer':r'$\hat{H}^{0} = \sum_{\tau, l_1, k_1} H^{0}_{\tau, l_1, \tau, l_1}(k_1, k_1) - \sum_{\tau, l_1, l_2, k_1, k_2} b_{\tau, l_1}^\dagger(k_1) (H^{0}_{\tau, l_1, \tau, l_2}(k_1, k_2))^* b_{\tau, l_2}(k_2)$'
}


In [436]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify the $\hat{H}^{0}$ in the hole basis.  
You should use canonical commutator relation for fermions to reorder the hole operator to the normal order. Normal order means that creation operators always appear before the annihilation operators.  You should follow the EXAMPLE below to simplify it to the normal order.  
Express the $\hat{H}^{0}$ in the normal order of $b_{l,\tau}(k)$ and also make $l_1$/$k_1$ always appear before $l_2$/$k_2$ in the index of $b$ and $H$.  
You should recall that $\hat{H}^{0} = \sum_{\tau, l_1, l_2, k_1, k_2} b_{\tau, l_1}(k_1) H^{0}_{\tau, l_1, \tau, l_2}(k_1, k_2) b_{\tau, l_2}^\dagger(k_2)$
Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
$\delta_{i,j}$ : Kronecker delta function, take the value of 1 if $i=j$, and 0 otherwise

In [437]:
kwargs_list.append(kwargs)

## Construct interaction Hamiltonian (momentum space)

In [438]:
kwargs = {
    'task':'Construct interaction Hamiltonian (momentum space)',
    'second_int_symbol':  r'$\hat{H}^{int}$',
    'index': r'valley index $\tau_i$ and layer index $l_i$',
    'momentum': r'momentum $k_i$',
    'For each operator, the total momentum is the sum of moire reciprocal lattice $b_i$ and momentum with in the first BZ $k_i$|None': '',
    'interaction': r'the dual-gate-screened Coulomb interaction',
    'int_form': r'$2\pi e^2\tanh(|q| d)/(\epsilon |q|)$',
    'normalization_factor': r'$\frac{1}{2 V}$',
    'op': r'$b_{l,\tau}(k)$',
    'def_var': r'''$d$ : distance between dual gates  
$\epsilon$ : dielectric constant  
$V(q)$ : dual-gate-screened Coulomb interaction in the momentum space''',
    'answer':r'$\hat{H}^{int} = \frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} b_{l_1,\tau_1}^\dagger(k_1)b_{l_2,\tau_2}^\dagger(k_2)b_{l_2,\tau_2}(k_3)b_{l_1,\tau_1}(k_4) V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$'
}


In [439]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to construct the interaction part of the Hamiltonian $\hat{H}^{int}$ in the momentum space.  
The interaction Hamiltonian is a product of four parts.
The first part is the product of four operators with two creation and two annihilation operators following the normal order, namely, creation operators are before annihilation operators. You should follow the order of $1,2,2,1$ for the valley index $\tau_i$ and layer index $l_i$, and $1,2,3,4$ for the momentum $k_i$. 
The second part is the constraint of total momentum conservation, namely the total momentum of all creation operators should be the same as that of all annihilation operators.   
The third part is the interaction form. You should use the dual-gate-screened Coulomb interaction with $V(q)=$2\pi e^2\tanh(|q| d)/(\epsilon |q|)$$, where $q$ is the transferred total momentum between a creation operator and an annilation operator with the same valley index $\tau_i$ and layer index $l_i$, namely $q=k_1-k_4$.  

In [440]:
kwargs_list.append(kwargs)

## Wick's theorem

In [441]:
kwargs = {
    'task':"Wick's theorem",
    'second_int_symbol':  r'$\hat{H}^{int}$',
    'HF_symbol': r'$\hat{H}^{int,HF}$',
    'expression_int': r'$\hat{H}^{int} = \frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} b_{l_1,\tau_1}^\dagger(k_1)b_{l_2,\tau_2}^\dagger(k_2)b_{l_2,\tau_2}(k_3)b_{l_1,\tau_1}(k_4) V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$',
    'def_var': r'''$\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_2) \rangle$ : expectation of $b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_2)$ over the many-body ground states.  
$\hat{H}^{int,HF}$ : Interaction term after Hartree-Fock approximation''',
    'answer':r'$\hat{H}^{int,HF}=\frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) + \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4)-\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle - \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) - \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) + \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle\right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$'
}


In [442]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to perform a Hartree-Fock approximation to expand the interaction term $\hat{H}^{int}$.  
You should use Wick's theorem to expand the four-fermion term in $\hat{H}^{int}$ into quadratic terms. You should strictly follow the EXAMPLE below to expand using Wick's theorem, select the correct EXAMPLE by noticing the order of four term product with and without ${}^\dagger$, and be extremely cautious about the order of the index and sign before each term.  
You should only preserve the normal terms. Here, the normal terms mean the product of a creation operator and an annihilation operator.  
You should recall that $\hat{H}^{int} = \frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} b_{l_1,\tau_1}^\dagger(k_1)b_{l_2,\tau_2}^\dagger(k_2)b_{l_2,\tau_2}(k_3)b_{l_1,\tau_1}(k_4) V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$.
Return the expanded interaction term after Hartree-Fock approximation $\hat{H}^{int,HF}$.

Use the following conventions for the symbols (You should also o

In [443]:
kwargs_list.append(kwargs)

## Extract quadratic term

In [444]:
kwargs = {
    'task':"Extract quadratic term",
    'HF_symbol': r'$\hat{H}^{int,HF}$',
    'bilinear_op': r'$b^\dagger b$',
    'HF_2_symbol': r'$\hat{H}^{int,HF,2}$',
    'expression_HF': r'$\hat{H}^{int,HF}=\frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) + \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4)-\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle - \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) - \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) + \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle\right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$',
    'def_var': r'''$\hat{H}^{int,HF,2}$ : the quadratic terms in $\hat{H}^{int,HF}$''',
    'answer':r'$\hat{H}^{int,HF,2}=\frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) + \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) - \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) - \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3)\right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$'
}


In [445]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to extract the quadratic terms in the $\hat{H}^{int,HF}$.  
The quadratic terms mean terms that are proportional to $b^\dagger b$, which excludes terms that are solely expectations or products of expectations.  
You should only preserve the quadratic terms in $\hat{H}^{int,HF}$, which is called $\hat{H}^{int,HF,2}$.  
You should recall that $\hat{H}^{int,HF}=\frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) + \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4)-\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle - \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) - \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle b_{l_1,\tau_1}^\dag

In [446]:
kwargs_list.append(kwargs)

## Swap the index to combine Hartree and Fock terms

In [447]:
kwargs = {
    'task':"Swap the index to combine Hartree and Fock terms",
    'HF_2_symbol': r'$\hat{H}^{int,HF,2}$',
    'expval': r'$\langle b_{\tau_1,l_1}^\dagger(k_1) b_{\tau_1,l_1}(k_4) \rangle$',
    'expression_Hartree_1':r'$\langle b_{\tau_1,l_1}^\dagger(k_1) b_{\tau_1,l_1}(k_4) \rangle b_{\tau_2,l_2}^\dagger(k_2) b_{\tau_2,l_2}(k_3)$',
    'expression_Hartree_2':r'$\langle b_{\tau_2,l_2}^\dagger(k_2) b_{\tau_2,l_2}(k_3) \rangle b_{\tau_1,l_1}^\dagger(k_1) b_{\tau_1,l_1}(k_4)$',
    'expression_HF_2':r'$\hat{H}^{int,HF,2}=\frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) + \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) - \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) - \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3)\right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$',
    'answer':r'$\hat{H}^{int,HF,2}=\frac{1}{V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) - \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4)\right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$'
}


In [448]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify the quadratic term $\hat{H}^{int,HF,2}$ through relabeling the index to combine the two Hartree/Fock term into one Hartree/Fock term.  
The logic is that the expected value ($\langle b_{\tau_1,l_1}^\dagger(k_1) b_{\tau_1,l_1}(k_4) \rangle$) in the first Hartree term ($\langle b_{\tau_1,l_1}^\dagger(k_1) b_{\tau_1,l_1}(k_4) \rangle b_{\tau_2,l_2}^\dagger(k_2) b_{\tau_2,l_2}(k_3)$) has the same form as the qudratic operators in the second Hartree term ($\langle b_{\tau_2,l_2}^\dagger(k_2) b_{\tau_2,l_2}(k_3) \rangle b_{\tau_1,l_1}^\dagger(k_1) b_{\tau_1,l_1}(k_4)$), and vice versa. The same applies to the Fock term.  
This means, if you relabel the index by swapping the index in the "expected value" and "quadratic operators" in the second Hartree term, you can make the second Hartree term look identical to the first Hartree term, as long as $V(q)=V(-q)$, which is naturally satisfied in Coulomb interaction. You should follow the EXAMPLE below to simplify

In [449]:
kwargs_list.append(kwargs)

In [450]:
len(kwargs_list)

13

In [451]:
--

SyntaxError: invalid syntax (3659366440.py, line 1)

## Save jsonl

In [452]:
with open('2111.01152.jsonl', 'w') as f:
    for dict_obj in kwargs_list:
        f.write(json.dumps(dict_obj) + '\n')

# Draft